In [7]:
from openai import OpenAI
import os
from dotenv import load_dotenv
from typing import List, Dict, Any
import httpx
from datetime import datetime
import re
import json

load_dotenv()

True

In [8]:
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
)

### Tool Definition Formatting
For me to use a tool, you need to define it in the following JSON format. This specifies the tool's name, description, and input parameters (using JSON Schema):

```json
{
  "name": "tool_name",
  "description": "Clear explanation of when to use this tool",
  "parameters": {
    "type": "object",
    "properties": {
      "param1": {
        "type": "string/number/boolean/array/object",
        "description": "Purpose of this parameter"
      },
      "param2": { ... }
    },
    "required": ["param1", ...],
    "additionalProperties": false // optional
  }
}
```

#### Key Notes:
1. **`name`**: Unique identifier for the tool (e.g., `get_weather`).  
2. **`description`**: Explains **when** the tool should be used (critical for accurate invocation).  
3. **`parameters`**: Defines inputs using [JSON Schema](https://json-schema.org/):
   - `type`: Data type (`string`, `number`, `boolean`, `array`, `object`).  
   - `description`: Clarifies the parameter's purpose.  
   - `required`: List mandatory parameters.  
   - `additionalProperties: false`: Disallows extra parameters (recommended).  

---

### Output Format for Tool Calling
When I decide to use a tool, I will output a JSON object in this exact structure:

```json
{
  "tool_name": "name_of_tool",
  "arguments": {
    "param1": "value1",
    "param2": "value2"
  }
}
```

#### Key Notes:
1. **`tool_name`**: Must match the defined tool name.  
2. **`arguments`**: Contains **only** the parameters defined in the tool's schema.  
   - Values respect data types (e.g., strings in quotes, numbers without quotes).  
   - Omit optional parameters not provided by the user.  
3. **No extra text**: The output is *pure JSON* without explanations or markdown.  

---

### Example Workflow
#### 1. Tool Definition (You Provide):
```json
{
  "name": "get_stock_price",
  "description": "Retrieve the current stock price for a given company ticker symbol.",
  "parameters": {
    "type": "object",
    "properties": {
      "ticker": {
        "type": "string",
        "description": "Stock ticker symbol (e.g., AAPL, TSLA)"
      },
      "currency": {
        "type": "string",
        "description": "Currency for conversion (optional, defaults to USD)"
      }
    },
    "required": ["ticker"],
    "additionalProperties": false
  }
}
```

#### 2. My Tool Call Output:
```json
{
  "tool_name": "get_stock_price",
  "arguments": {
    "ticker": "TSLA",
    "currency": "EUR"
  }
}
```

#### 3. Tool Execution (You Run):
You call `get_stock_price(ticker="TSLA", currency="EUR")` → Return result (e.g., `{"price": 210.50}`).  

#### 4. Final Response (I Generate):
"Tesla's current stock price is €210.50."

---

### Important Notes:
- **Single Tool per Output**: I invoke one tool at a time.  
- **Validation**: I strictly follow your tool's schema (data types, required fields).  
- **Error Handling**: If arguments are invalid, I regenerate the call or handle errors gracefully.  

Let me know your tools, and I'll use them!  🚀

In [9]:
tools = [
    {
        "type": "function",
        "name": "is_statement_correct",
        "description": "Retrieve the statement is right or wrong",
        "parameters": {
            "type": "object",
            "properties": {
                "statement": {
                    "type": "string",
                    "description": "Statement for the user"
                }
            },
            "required": ["statement"],
        }
    }
]


def is_statement_correct(statement):
    return input(f"{statement} yes/no")

In [27]:
xiamoni_input_list = [
    {
        "role" : "system",
        "content" : """
        You are MiMo, an AI assistant developed by Xiaomi. Today's date: 04 January 2026. Your knowledge cutoff date is December 2024. 
        You will act like a profession guesser for the user. For doing this you will define short statement about the user and ask the user whether the statement is right or wrong using tools.
        Try to make the statement concise not more than 10 words.
        Finish the task within 2 observation and reply Answer with this format "Profession".
        """
    },
    {
        "role" : "user",
        "content" : "Can you guess my profession?"
    }
]

In [ ]:
input_list = [
    {
        "role" : "developer",
        "content" : """
        You are a profession guesser assistant.You will act like a profession guesser for the user. For doing this you will define short statement about the user and ask the user whether the statement is right or wrong using tools.
        Try to make the statement concise not more than 10 words.
        Finish the task within 10 observation and reply Answer with this format "Profession".
        """
    },
    {
        "role" : "user",
        "content" : "Can you guess my profession?"
    }
]

In [ ]:
cnt = 0
while cnt <= 10:
    response = client.responses.create(
        model="openai/gpt-oss-120b:free",
        tools=tools,
        input=input_list,
    )

    input_list += response.output
    final_answer = False
    for item in response.output:
        if item.type == "function_call":
            if item.name == "is_statement_correct":
                # 3. Execute the function logic for is_statement_correct
                isCorrect = is_statement_correct(json.loads(item.arguments))
                
                # 4. Provide function call results to the model
                input_list.append({
                    "type": "function_call_output",
                    "call_id": item.call_id,
                    "output": json.dumps({
                        "isCorrect": isCorrect
                    })
                })
        elif item.type == "message":
            final_answer = True
            break
    cnt += 1
    print(input_list)
    if final_answer:
        print(response.output_text)
        break

    

[{'role': 'developer', 'content': '\n        You are a profession guesser assistant.You will act like a profession guesser for the user. For doing this you will define short statement about the user and ask the user whether the statement is right or wrong using tools.\n        Try to make the statement concise not more than 10 words.\n        Finish the task within 2 observation and reply Answer with this format "Profession".\n        '}, {'role': 'user', 'content': 'Can you guess my profession?'}, ResponseReasoningItem(id='rs_tmp_ws3ro4cj4je', summary=[], type='reasoning', content=[Content(text='We need to guess profession by asking a statement and using tool to check if statement is right or wrong. We have up to 2 observations (i.e., two attempts). We need to define a short statement about the user (max 10 words) and ask the user whether the statement is right or wrong using tools. The tool is is_statement_correct which takes a statement and returns something? It says "Retrieve the s

In [19]:
input_list

[{'role': 'developer',
  'content': '\n        You are a profession guesser assistant.You will act like a profession guesser for the user. For doing this you will define short statement about the user and ask the user whether the statement is right or wrong using tools.\n        Try to make the statement concise not more than 10 words.\n        Finish the task within 10 observation and reply Answer with this format "Profession".\n        '},
 {'role': 'user', 'content': 'Can you guess my profession?'},
 ResponseReasoningItem(id='rs_tmp_5cnbpuxk6fx', summary=[], type='reasoning', content=[Content(text='We need to guess profession by asking up to 10 observations. We need to define short statement about the user and ask if it\'s right or wrong using tool is_statement_correct. So we generate a statement, ask tool to check? The tool expects statement and returns something? Probably returns true/false. We need to iterate. We\'ll ask a statement, get response, then adjust. Finally after up to

In [ ]:
response = client.chat.completions.create(
  model="xiaomi/mimo-v2-flash:free",
  messages=[
          {
            "role": "user",
            "content": "What is the tool defination formatting for you? and What will be the output format for tool calling?"
          }
        ],
  extra_body={"reasoning": {"enabled": True}}
)

response = response.choices[0].message
print(response.content)

### Tool Definition Formatting
For me to use a tool, you need to define it in the following JSON format. This specifies the tool's name, description, and input parameters (using JSON Schema):

```json
{
  "name": "tool_name",
  "description": "Clear explanation of when to use this tool",
  "parameters": {
    "type": "object",
    "properties": {
      "param1": {
        "type": "string/number/boolean/array/object",
        "description": "Purpose of this parameter"
      },
      "param2": { ... }
    },
    "required": ["param1", ...],
    "additionalProperties": false // optional
  }
}
```

#### Key Notes:
1. **`name`**: Unique identifier for the tool (e.g., `get_weather`).  
2. **`description`**: Explains **when** the tool should be used (critical for accurate invocation).  
3. **`parameters`**: Defines inputs using [JSON Schema](https://json-schema.org/):
   - `type`: Data type (`string`, `number`, `boolean`, `array`, `object`).  
   - `description`: Clarifies the parameter's pur